In [3]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
# from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options
import re

directory = pd.read_json('directory.json').sort_index()
directory.loc[directory['more'].notnull(),'more'].index

Int64Index([   0,    1,    2,    9,   10,   15,   16,   19,   20,   22,
            ...
            1513, 1514, 1515, 1516, 1517, 1519, 1523, 1525, 1528, 1531],
           dtype='int64', length=471)

In [43]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
# from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options
import re

# %matplotlib inline

directory = pd.read_json('directory.json').sort_index()
documents = pd.read_json('documents.json')


breaks=[
    [0,100],
    [100,200],
    [200,300],
    [300,400],
    [400,500],
]

for z in breaks:
    options = Options()
    options.add_argument("--headless")
    options.add_argument('log-level=1')     
    driver = webdriver.Chrome(options=options, executable_path="chromedriver.exe")
    print("Chrome Headless Browser Invoked")

    try:
        subdirectory_progress = pd.read_json('subdirectory_progress.json')
    except:
        subdirectory_progress = pd.DataFrame(columns=['sub_url'])

    try:
        directory_progress = pd.read_json('directory_progress.json')
    except:
        directory_progress = pd.DataFrame(columns=['dir_url'])


    counter = 0
    dir_counter = 0
    problems = []



    for i in directory.loc[directory['more'].notnull(),'more'].index[z[0]:z[1]]:
        try:
            url_root = directory.loc[i,'url']
            if url_root in set(directory_progress['dir_url']):
                print('SKIP DIR')
                continue
            else:
                print('\n','ROOT')
                print(url_root)
                print('\n','SUBS')
                country = directory.loc[i,'country']
                page = directory.loc[i,'page']
                lang = directory.loc[i,'lang']

                subpage_count = int(re.split(r'_|\.',directory.loc[i,'more'])[1])
                subpages = np.arange(subpage_count)+1
                subpages_urls = [(url_root+'default_'+str(m)+'.shtml') for m in subpages]

                dir_counter += 1

                for h in subpages_urls:
                    if h in set(subdirectory_progress['sub_url']):
                        print('SKIP SUB')
                        continue
                    else:                
                        print(h,'\n')
                        driver.get(h)
                        soup=BeautifulSoup(driver.page_source, 'lxml')

                        nav = str()
                        for k in soup.find(class_='nav').find_all('a'):
                            nav = nav + k.text + '_'

                #         try:
                #             more = soup.find("a", string='尾页')['href']
                #             print(more)
                #             directory.loc[i,'more'] = more
                #         except:
                #             print('NO MORE')

                        for j in soup.find(class_='rebox_news').find_all('li'):
                            title = j.text
                            url_new = url_root+j.find('a')['href'][2:]
                            print(url_new)

                            try:
                                date = re.findall(r"\d\d\d\d\-\d\d\-\d\d",j.text)[0]
                            except:
                                date = '9999-99-99'
                            print(date)

                            documents = documents.append({'country':country,
                                                          'page':page,
                                                          'url':url_new,
                                                          'lang':lang,
                                                          'nav':nav,
                                                          'title':title,
                                                          'date':date,
                                                         },
                                                         ignore_index=True)
                            subdirectory_progress = subdirectory_progress.append({'sub_url':h},ignore_index=True)
                            counter += 1
                            print(dir_counter)
                            print(counter,'\n')

                    directory_progress = directory_progress.append({'dir_url':url_root},ignore_index=True)
        except:
            problems.append(i)

    documents = documents.drop_duplicates()
    subdirectory_progress = subdirectory_progress.drop_duplicates()
    directory_progress = directory_progress.drop_duplicates()

    documents.to_json('documents.json')
    subdirectory_progress.to_json('subdirectory_progress.json')
    directory_progress.to_json('directory_progress.json')
    pd.Series(problems).to_json('problems.json')
    driver.quit()
    print(z)